In [1]:
import pandas as pd

In [2]:
champs = pd.read_csv('../output_etl/champs.csv', index_col=0)
matches = pd.read_csv('../output_etl/matches.csv', index_col=0)
participants = pd.read_csv('../output_etl/participants.csv', index_col=0)
stats = pd.read_csv('../output_etl/stats.csv', index_col=0)
bans = pd.read_csv('../output_etl/teambans.csv', index_col=0)

In [3]:
participants = participants [['matchid', 'player', 'championid', 'role', 'position']]

In [4]:
bans = bans[['matchid', 'championid', 'banturn']]

In [5]:
participants_win = participants.merge(stats, left_on='matchid', right_on='id', how='left')

In [6]:
df_win_ban = participants_win.merge(bans, left_on='matchid', right_on='matchid', how='left')

In [7]:
df_win_ban.rename(columns={'championid_x': 'pick', 'championid_y': 'ban'}, inplace=True)

In [8]:
df = df_win_ban[['pick', 'ban', 'position', 'win']]

In [9]:
df_model = df.dropna()

In [10]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9424320 entries, 0 to 10963463
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   pick      int64  
 1   ban       float64
 2   position  object 
 3   win       float64
dtypes: float64(2), int64(1), object(1)
memory usage: 359.5+ MB


### Feature Engineering

In [11]:
df_model = pd.get_dummies(df_model, columns=['pick', 'ban', 'position'])

In [12]:
df_model.win.value_counts()

win
1.0    4713058
0.0    4711262
Name: count, dtype: int64

### Modelagem

#### Random Forest

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [19]:
df_sample = df_model.sample(frac=0.01)

In [20]:
X = df_sample.drop('win', axis=1)
y = df_sample['win']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape

(75394, 276)

In [31]:
cols = X_train.columns
df_cols = pd.DataFrame(cols, columns=['cols'])
df_cols.to_csv('../models/cols.csv')

In [32]:
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [35]:
X_test

,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9,pick_10,...,ban_421.0,ban_427.0,ban_429.0,ban_432.0,ban_497.0,ban_498.0,position_BOT,position_JUNGLE,position_MID,position_TOP
8497031,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
5972068,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2671472,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
10520493,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
10740643,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6587328,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
8489537,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4421430,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7914734,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [33]:
# Calcule a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

# Mostrar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusão:')
print(conf_matrix)

# Mostrar o relatório de classificação
class_report = classification_report(y_test, y_pred)
print('Relatório de Classificação:')
print(class_report)

Acurácia: 0.49864714308451374
Matriz de Confusão:
[[4621 4859]
 [4591 4778]]
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.50      0.49      0.49      9480
         1.0       0.50      0.51      0.50      9369

    accuracy                           0.50     18849
   macro avg       0.50      0.50      0.50     18849
weighted avg       0.50      0.50      0.50     18849



#### Multi-modelo

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Suponha que X e y são seus dados de entrada e saída

# Divida os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos
models = {
    #'Random Forest': RandomForestClassifier(n_estimators=10, random_state=42),
    #'SVM': SVC(kernel='linear', random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=10, random_state=42)
}

# Treine e avalie cada modelo
for model_name, model in models.items():
    print(f"=== {model_name} ===")
    
    # Treine o modelo
    model.fit(X_train, y_train)
    
    # Faça previsões no conjunto de teste
    y_pred = model.predict(X_test)
    
    # Calcule a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Acurácia: {accuracy}')
    
    # Mostrar a matriz de confusão
    conf_matrix = confusion_matrix(y_test, y_pred)
    print('Matriz de Confusão:')
    print(conf_matrix)
    
    # Mostrar o relatório de classificação
    class_report = classification_report(y_test, y_pred)
    print('Relatório de Classificação:')
    print(class_report)
    
    # Avaliação usando validação cruzada
    cv_accuracy = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f'Acurácia Média (Cross-Validation): {cv_accuracy.mean()}')
    
    print('\n')


=== Gradient Boosting ===
Acurácia: 0.4994959944824659
Matriz de Confusão:
[[9041  349]
 [9085  374]]
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.50      0.96      0.66      9390
         1.0       0.52      0.04      0.07      9459

    accuracy                           0.50     18849
   macro avg       0.51      0.50      0.37     18849
weighted avg       0.51      0.50      0.36     18849

Acurácia Média (Cross-Validation): 0.5033052776191737




Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

# Suponha que X e y são seus dados de entrada e saída

# Divida os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie e ajuste o modelo Bernoulli Naive Bayes
nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = nb_model.predict(X_test)

# Calcule a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

# Mostrar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusão:')
print(conf_matrix)

# Mostrar o relatório de classificação
class_report = classification_report(y_test, y_pred)
print('Relatório de Classificação:')
print(class_report)


Acurácia: 0.5011936972783702
Matriz de Confusão:
[[5117 4273]
 [5129 4330]]
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.50      0.54      0.52      9390
         1.0       0.50      0.46      0.48      9459

    accuracy                           0.50     18849
   macro avg       0.50      0.50      0.50     18849
weighted avg       0.50      0.50      0.50     18849



Rede Neural

In [36]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Suponha que X e y são seus dados de entrada e saída
# Certifique-se de ajustar o número de features em input_dim

# Divida os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construa o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid para classificação binária
])

# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treine o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Avalie o modelo no conjunto de teste
y_pred_probs = model.predict(X_test)
y_pred = tf.round(y_pred_probs)  # Arredonda as previsões para 0 ou 1

# Calcule a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

# Mostrar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusão:')
print(conf_matrix)

# Mostrar o relatório de classificação
class_report = classification_report(y_test, y_pred)
print('Relatório de Classificação:')
print(class_report)





Epoch 1/10


1885/1885 [==============================] - 4s 2ms/step - loss: 0.6933 - accuracy: 0.4972 - val_loss: 0.6935 - val_accuracy: 0.4980
Epoch 2/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6928 - accuracy: 0.5107 - val_loss: 0.6935 - val_accuracy: 0.4994
Epoch 3/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6916 - accuracy: 0.5234 - val_loss: 0.6947 - val_accuracy: 0.4961
Epoch 4/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6897 - accuracy: 0.5331 - val_loss: 0.6980 - val_accuracy: 0.5026
Epoch 5/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6873 - accuracy: 0.5454 - val_loss: 0.6995 - val_accuracy: 0.5004
Epoch 6/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6853 - accuracy: 0.5481 - val_loss: 0.7007 - val_accuracy: 0.5012
Epoch 7/10
1885/1885 [==============================] - 3s 1ms/step - loss: 0.6828 - accuracy: 0.5562 - val_loss: 0.7026 - val_accu

In [ ]:
from sklearn.linear_model import LogisticRegression

# Suponha que X e y são seus dados de entrada e saída

# Divida os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie e ajuste o modelo de regressão logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = logistic_model.predict(X_test)

# Calcule a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

# Mostrar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusão:')
print(conf_matrix)

# Mostrar o relatório de classificação
class_report = classification_report(y_test, y_pred)
print('Relatório de Classificação:')
print(class_report)


Acurácia: 0.5024669743752984
Matriz de Confusão:
[[5221 4169]
 [5209 4250]]
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.50      0.56      0.53      9390
         1.0       0.50      0.45      0.48      9459

    accuracy                           0.50     18849
   macro avg       0.50      0.50      0.50     18849
weighted avg       0.50      0.50      0.50     18849



### Encapsulamento

In [37]:
import pickle
from datetime import datetime

# Obtenha a data e hora atual
current_datetime = datetime.now()

# Formate a data e hora como uma string
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# Construa o nome do arquivo com a data e hora
file_name = f'../models/model_{formatted_datetime}.pkl'

# Salve o modelo em um arquivo pkl
with open(file_name, 'wb') as model_file:
    pickle.dump(model, model_file)


In [ ]:
model_csv = X_train.iloc[0]
model_csv.to_csv('../models/model2.csv', index=True)

In [ ]:
pd.set_option('display.max_rows', None)
X_train.iloc[0]

pick_1             False
pick_2             False
pick_3             False
pick_4             False
pick_5             False
pick_6             False
pick_7             False
pick_8             False
pick_9             False
pick_10            False
pick_11            False
pick_12            False
pick_13            False
pick_14            False
pick_15            False
pick_16            False
pick_17            False
pick_18            False
pick_19            False
pick_20            False
pick_21            False
pick_22            False
pick_23            False
pick_24            False
pick_25            False
pick_26            False
pick_27            False
pick_28            False
pick_29            False
pick_30            False
pick_31            False
pick_32            False
pick_33            False
pick_34            False
pick_35             True
pick_36            False
pick_37            False
pick_38            False
pick_39            False
pick_40            False
